# Atualizar os nomes dos quartos em um modelo

Existem muitas situações em que você gostaria de atualizar as informações em um arquivo IFC. Um caso pode ser a necessidade de atualizar informações sobre quartos em um modelo.

As salas de um modelo ifc são do tipo [IfcSpace](https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/FINAL/HTML/ifcproductextension/lexical/ifcspace.htm) e existem muitas maneiras possíveis de adicionar informações a ele como atributos diretos e em conjuntos de propriedades predefinidas ou definidas de outra forma.

Como o **IfcSpace** herda de [IfcRoot](https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifckernel/lexical/ifcroot.htm), ele tem atributos diretos de **Name**, **Description**, **GlobalID** e além disso, o [IfcSpatialStructureElement](https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/FINAL/HTML/ifcproductextension/lexical/ifcspatialstructureelement.htm) também tem o atributo **LongName**.

Neste caso, estamos interessados em obter o Name, LongName e GlobalID de todos os quartos no modelo em um formato editável para verificar e atualizar os nomes dos quartos.

Então, vamos percorrer

1. Como obter um arquivo ifc, consultar sua sala e os atributos diretos de Name, LongName e GlobalID
2. Grave isso em um arquivo csv para manipular em um editor de terceiros como, por exemplo. Planilha do Google ou Excel.
3. A seguir, mostramos como ler essas informações de um arquivo csv (separado por ",") e atualizar o modelo IFC.

## 1. Obtendo os quartos e atributos da IFC

In [2]:
# importar biblioteca ifcopenshell e abrir arquivo ifc usando ifcopenshell

import ifcopenshell 

file = ifcopenshell.open("Grethes-hus-bok-2.ifc")

In [8]:
# Obtém uma referência para todos os objetos no arquivo que são do tipo "IfcSpace", obtendo efetivamente a 
# lista de todos os quartos no modelo.

rooms = file.by_type("IfcSpace")

print(f"Exitem {len(rooms)} salas neste modelo")

Exitem 8 salas neste modelo


In [9]:
## Percorra as salas e imprima o valor de seus atributos para fins de visualização intermediária.

for room in rooms:                                        
    print(f"""
    Nome da sala (room):\t\t {room.Name}
    LongName:\t\t {room.LongName}
    Descrição da sala: {room.Description}\t\t
    GUID da sala: {room.GlobalId}""")


    Nome da sala (room):		 1
    LongName:		 Kjøkken øst
    Descrição da sala: None		
    GUID da sala: 1gY9E27$nCk8ELruXgAK4X

    Nome da sala (room):		 2
    LongName:		 Stue øst
    Descrição da sala: None		
    GUID da sala: 1gY9E27$nCk8ELruXgAK4k

    Nome da sala (room):		 3
    LongName:		 Bod øst
    Descrição da sala: None		
    GUID da sala: 1gY9E27$nCk8ELruXgAK4h

    Nome da sala (room):		 4
    LongName:		 Bad øst
    Descrição da sala: None		
    GUID da sala: 1gY9E27$nCk8ELruXgAK4e

    Nome da sala (room):		 5
    LongName:		 Bad vest
    Descrição da sala: None		
    GUID da sala: 1gY9E27$nCk8ELruXgAK3L

    Nome da sala (room):		 6
    LongName:		 Bod vest
    Descrição da sala: None		
    GUID da sala: 1gY9E27$nCk8ELruXgAK3I

    Nome da sala (room):		 7
    LongName:		 Stue vest
    Descrição da sala: None		
    GUID da sala: 1gY9E27$nCk8ELruXgAK3V

    Nome da sala (room):		 8
    LongName:		 Kjøkken vest
    Descrição da sala: None		
    GUID da sala: 1gY9E27$n

## 2. Escreva os atributos GlobalID, Name e LongName de todas as salas em csv

In [10]:
## Grave GUID, NAME, LongName de salas em um arquivo csv para uso em outros aplicativos

import csv

with open('rooms.csv', 'w') as csvfile:
    fieldnames = ['GUID', 'Name','LongName','NameToUpdate']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for room in rooms:
        writer.writerow({'GUID': room.GlobalId, 'Name': room.Name, 'LongName': room.LongName,'NameToUpdate':''})


Documentação » A biblioteca padrão do Python
    
Mais: [csv - Leitura e gravação de arquivo CSV] (https://docs.python.org/3/library/csv.html)

## 3. Leia o conteúdo de um arquivo csv para atualizar o nome e o nome longo das salas dos GlobalIDs fornecidos

In [11]:
## Crie uma lista para conter cada objeto do quarto que mais tarde será adicionado ao modelo
## Isso assume um formato semelhante ao csv que foi escrito.
# Para sua informação: este arquivo de exemplo é editado e exportado para csv novamente ("," delimitado) usando a 
# planilha do Google
# Outros aplicativos podem exportar com outra formatação e/ou codificação

list_of_new_rooms = []

with open('rooms.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        list_of_new_rooms.append(row)
        print(row['GUID'], row['Name'],row['LongName'])  

1gY9E27$nCk8ELruXgAK4X 1 Kjøkken øst
1gY9E27$nCk8ELruXgAK4k 2 Stue øst
1gY9E27$nCk8ELruXgAK4h 3 Bod øst
1gY9E27$nCk8ELruXgAK4e 4 Bad øst
1gY9E27$nCk8ELruXgAK3L 5 Bad vest
1gY9E27$nCk8ELruXgAK3I 6 Bod vest
1gY9E27$nCk8ELruXgAK3V 7 Stue vest
1gY9E27$nCk8ELruXgAK3S 8 Kjøkken vest


In [12]:
## Use file.by_guid("GlobalID") para obter os objetos exatos do arquivo e atualizar seus atributos do arquivo.
## Isso pega a sala por seu guid do arquivo, em seguida, atualiza seus atributos Name e LongName 

for room in list_of_new_rooms:
    room_by_guid = file.by_guid(room["GUID"])
    room_by_guid.Name = room['Name']
    room_by_guid.LongName = room['LongName']

In [13]:
## Agora os atributos Name e LongName dos espaços foram atualizados. Em seguida, 
# você pode imprimir uma nova versão do arquivo ifc.
## Agora o arquivo está armazenado em sua pasta de início. 

new_file_name = "new_kontorbyggDIBK.ifc"
file.write(new_file_name)

# Extra - verifique IfcCSV!

[BlenderBIM IFCCSV](https://wiki.osarch.org/index.php?title=BlenderBIM_IFCCSV) - O recurso IFC CSV do IfcOpenShell é implementado no complemento BlenderBIM

Tour do BlenderBIM com Dion Moult - IFCCSV - [vídeo](https://www.youtube.com/watch?v=ZeMkG6jTzAg)

<img src="blender.png">